In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
raw_file = "/Users/wenfengzeng/data/multinorch/20240203_Ecl1_Evo08_11p5min_TiHe_SA_H032_E14_B5.raw"
from alpharaw.thermo import ThermoRawData

raw_data = ThermoRawData(
    process_count=1,
    auxiliary_items=["multinotch"],
)
raw_data.import_raw(raw_file)
raw_data.spectrum_df

,spec_idx,peak_start_idx,peak_stop_idx,rt,precursor_mz,precursor_charge,isolation_lower_mz,isolation_upper_mz,ms_level,nce,multinotch
0,0,0,2864,4.108146,452.239471,0,452.039471,452.439471,2,30.0,"[[452.03947143256664, 452.4394714385271], [457..."
1,1,2864,5606,4.116757,452.239471,0,452.039471,452.439471,2,30.0,"[[452.03947143256664, 452.4394714385271], [457..."
2,2,5606,8357,4.125367,452.239471,0,452.039471,452.439471,2,30.0,"[[452.03947143256664, 452.4394714385271], [457..."
3,3,8357,11260,4.133977,452.239471,0,452.039471,452.439471,2,30.0,"[[452.03947143256664, 452.4394714385271], [457..."
4,4,11260,13770,4.142586,452.239471,0,452.039471,452.439471,2,30.0,"[[452.03947143256664, 452.4394714385271], [457..."
...,...,...,...,...,...,...,...,...,...,...,...
665,665,1541227,1542980,10.599350,770.464478,0,770.264478,770.664478,2,30.0,"[[770.2644775360823, 770.6644775420427], [775...."
666,666,1542980,1544857,10.608726,770.464478,0,770.264478,770.664478,2,30.0,"[[770.2644775360823, 770.6644775420427], [775...."
667,667,1544857,1546730,10.618007,770.464478,0,770.264478,770.664478,2,30.0,"[[770.2644775360823, 770.6644775420427], [775...."
668,668,1546730,1548591,10.627298,770.464478,0,770.264478,770.664478,2,30.0,"[[770.2644775360823, 770.6644775420427], [775...."


In [3]:
spec_df = raw_data.spectrum_df
peak_df = raw_data.peak_df

In [4]:
from alphabase.constants.modification import MOD_DF
MOD_DF[MOD_DF.mod_name=="Label:13C(6)15N(4)@R"]

,mod_name,unimod_mass,unimod_avge_mass,composition,unimod_modloss,modloss_composition,classification,unimod_id,modloss_importance,mass,modloss_original,modloss
mod_name,,,,,,,,,,,,
Label:13C(6)15N(4)@R,Label:13C(6)15N(4)@R,10.008269,9.9296,C(-6)13C(6)N(-4)15N(4),0.0,,Isotopic label,267,0.0,10.008269,0.0,0.0


In [5]:
from alpharaw.viz.psm_plot import PSM_Plot
from alpharaw.viz.xic_plot import XIC_Plot
from alpharaw.viz.psm_plot import plot_multi_spectra, plot_multi_psms
from alpharaw.viz.df_utils import make_psm_plot_df_for_peptide, make_query_df_for_peptide
import numpy as np

xic_plotter = XIC_Plot()
psm_plotter = PSM_Plot()

def plot_xic(
    spec_df, peak_df,
    query_masses, rt_sec,
    query_ion_names,
    precursor_mz,
    ms1_mz_tol = 20,
    title="",
):
    if precursor_mz > 0:
        precursor_left_mz = precursor_mz*(1-ms1_mz_tol*1e-6)
        precursor_right_mz = precursor_mz*(1+ms1_mz_tol*1e-6)
    else:
        precursor_left_mz = -1.0
        precursor_right_mz = -1.0
    return xic_plotter.plot_query_masses(
        spec_df, peak_df,
        query_masses,
        query_ion_names=query_ion_names,
        query_rt_sec=rt_sec, 
        precursor_left_mz=precursor_left_mz,
        precursor_right_mz=precursor_right_mz,
        query_intensities=None,
        title=title,
    )

def plot_multi_psm_xic(
    spec_df, peak_df,
    sequences, mods_list, 
    mod_sites_list, charges,
    rt_secs, include_isotopes=False
):
    plot_df_list = []
    for i, (seq, mods, mod_sites, charge, rt_sec) in enumerate(zip(
        sequences, mods_list, mod_sites_list, charges, rt_secs
    )):
        plot_df = make_query_df_for_peptide(
            seq, mods, mod_sites, charge, rt_sec,
            ms_level=2, include_precursor_isotopes=include_isotopes,
        )
        xic_plotter.plot(
            spec_df, peak_df,
            plot_df, title=plot_df.modified_sequence.values[0],
            add_peak_area=True, 
            create_new_fig=len(plot_df_list)==0,
            plot_rows = len(sequences),
            ith_plot_row=i
        )
        plot_df_list.append(plot_df)
    return plot_df_list, xic_plotter.fig

def plot_one_psm_xic(
    spec_df, peak_df,
    sequence, mods, 
    mod_sites, charge,
    rt_sec, include_isotopes=False
):
    plot_df_list, fig = plot_multi_psm_xic(
        spec_df, peak_df, 
        [sequence], [mods], [mod_sites], [charge],
        [rt_sec], include_isotopes=include_isotopes,
    )
    return plot_df_list[0], fig

def plot_one_psm(
    peak_df,
    sequence, mods, mod_sites, charge,
):
    plot_df = make_psm_plot_df_for_peptide(
        peak_df.mz.values,
        peak_df.intensity.values,
        sequence, mods, mod_sites, charge
    )
    return psm_plotter.plot(
        plot_df, sequence, 
        plot_df.modified_sequence.values[0],
        plot_unmatched_peaks=True
    )

def plot_psms(
    peak_df_list,
    sequence, mods, mod_sites, charge,
    plot_height=800
):
    return plot_multi_psms(
        [_df.mz.values for _df in peak_df_list],
        [_df.intensity.values for _df in peak_df_list],
        sequence, mods, mod_sites, charge, 
        plot_height=plot_height
    )

def plot_matches(
    peak_df_list,
    query_masses, 
    ion_names,
    plot_height = 800
):
    spec_masses_list = []
    spec_intens_list = []
    for i in range(len(peak_df_list)):
        spec_masses = peak_df_list[i].mz.values
        _slices = (spec_masses>=query_masses.min()-3)&(spec_masses<=query_masses.max()+3)
        spec_masses_list.append(spec_masses[_slices])
        spec_intens_list.append(peak_df_list[i].intensity.values[_slices])
    return plot_multi_spectra(
        spec_masses_list, spec_intens_list, 
        query_masses, query_ion_names=ion_names,
        query_mass_tols=query_masses*10*1e-6,
        title="", plot_height=plot_height
    )

def lookup_multinotch(precursor_mz, multinotch_list):
    for left,right in multinotch_list:
        if left<=precursor_mz and right>=precursor_mz:
            return True
    return False

def find_neibor_spec_idxes(
    spec_df, rt_sec, 
    precursor_mz, n
):
    multinotch = "multinotch" in spec_df.columns
    spec_idx = np.searchsorted(spec_df.rt.values, rt_sec/60)
    left_idxes = []
    right_idxes = []
    for i in range(spec_idx, -1, -1):
        if len(left_idxes) > (n//2 if n%2 else n//2-1): break
        if multinotch:
            if lookup_multinotch(precursor_mz, spec_df.multinotch.values[i]):
                left_idxes.append(i)
        else:
            if (
                spec_df.isolation_lower_mz.values[i]<=precursor_mz
                and spec_df.isolation_upper_mz.values[i]>= precursor_mz
            ):
                left_idxes.append(i)
    for i in range(spec_idx+1, len(spec_df)):
        if len(right_idxes) >= n//2: break
        if (
            spec_df.isolation_lower_mz.values[i]<=precursor_mz
            and spec_df.isolation_upper_mz.values[i]>= precursor_mz
        ):
            right_idxes.append(i)
    return sorted(left_idxes+right_idxes)

def get_peak_df_list(
    spec_df, peak_df, 
    rt_sec, precursor_mz,
    n = 1
):
    peak_df_list = []
    spec_idxes = find_neibor_spec_idxes(
        spec_df, rt_sec, precursor_mz, n
    )
    for i in spec_idxes:
        start = spec_df.peak_start_idx.values[i]
        stop = spec_df.peak_stop_idx.values[i]
        peak_df_list.append(peak_df.iloc[start:stop,:])
    return peak_df_list

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [6]:
import pandas as pd
import alphabase.peptide.precursor as precursur_func
psm_df = pd.DataFrame({
    "sequence": ["MGALGLEGR","MGALGLEGR"],
    "charge": [2,2],
    "mods": ["","Label:13C(6)15N(4)@R"], 
    "mod_sites": ["","9"],
    "rt": [4.3,4.3],
})
precursur_func.calc_precursor_mz(psm_df)
precursur_func.calc_precursor_isotope(psm_df)

,sequence,charge,mods,mod_sites,rt,nAA,precursor_mz,i_0,i_1,i_2,i_3,i_4,i_5,mono_isotope_idx
0,MGALGLEGR,2,,,4.3,9,452.239470,0.590296,0.273583,0.101809,0.027392,0.005874,0.001045,0
1,MGALGLEGR,2,Label:13C(6)15N(4)@R,9,4.3,9,457.243604,0.000000,0.002749,0.060906,0.615891,0.235931,0.084523,0


In [7]:
i_psm = 0
psm_rt_sec = psm_df.rt.values[i_psm]*60
psm_mz = psm_df.precursor_mz.values[i_psm]
psm_seq = psm_df.sequence.values[i_psm]
psm_mods = psm_df.mods.values[i_psm]
psm_sites = psm_df.mod_sites.values[i_psm]
psm_charge = psm_df.charge.values[i_psm]
psm_rt_sec

258.0

In [8]:
xic_plotter.rt_sec_win = 100
plot_df, fig = plot_one_psm_xic(
    spec_df, peak_df, 
    psm_df.sequence.values[0],
    psm_df.mods.values[0], 
    psm_df.mod_sites.values[0],
    psm_df.charge.values[0],
    psm_df.rt.values[0]*60, 
    include_isotopes=False
)

In [9]:
xic_plotter.rt_sec_win = 100
plot_df_list, fig = plot_multi_psm_xic(
    spec_df, peak_df, 
    psm_df.sequence.values,
    psm_df.mods.values, 
    psm_df.mod_sites.values,
    psm_df.charge.values,
    psm_df.rt.values*60, 
    include_isotopes=False
)

In [10]:
one_peak_df = get_peak_df_list(
    spec_df, peak_df, psm_rt_sec, psm_mz, n=1
)[0]
fig = plot_one_psm(
    one_peak_df, psm_seq, psm_mods, psm_sites, psm_charge,
)

In [11]:
fig = plot_psms(
    get_peak_df_list(
        spec_df, peak_df,
        psm_rt_sec, 
        psm_mz,
        n=3
    ),
    psm_seq, psm_mods, psm_sites, psm_charge,
    plot_height=800
)